**CrossValidation y mejora de modelos de ML**

In [20]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score, LeaveOneOut, KFold, StratifiedKFold
from scipy.stats import uniform, randint

In [2]:
# Cargar el archivo con Pandas
url = "https://raw.githubusercontent.com/iosonobenjamin/Proyecto-Data-Science/main/Airline%20Dataset.csv"
df = pd.read_csv(url)

In [3]:
# Aplicar el one-hot encoding a la variable Flight Status
df = pd.get_dummies(df, columns=["Flight Status"])

# Mostrar las primeras filas del nuevo dataframe
df.head()

,Passenger ID,First Name,Last Name,Gender,Age,Nationality,Airport Name,Airport Country Code,Country Name,Airport Continent,Continents,Departure Date,Arrival Airport,Pilot Name,Flight Status_Cancelled,Flight Status_Delayed,Flight Status_On Time
0,10856,Edithe,Leggis,Female,62,Japan,Coldfoot Airport,US,United States,NAM,North America,6/28/2022,CXF,Edithe Leggis,False,False,True
1,43872,Elwood,Catt,Male,62,Nicaragua,Kugluktuk Airport,CA,Canada,NAM,North America,12/26/2022,YCO,Elwood Catt,False,False,True
2,42633,Darby,Felgate,Male,67,Russia,Grenoble-Isère Airport,FR,France,EU,Europe,1/18/2022,GNB,Darby Felgate,False,False,True
3,78493,Dominica,Pyle,Female,71,China,Ottawa / Gatineau Airport,CA,Canada,NAM,North America,9/16/2022,YND,Dominica Pyle,False,True,False
4,82072,Bay,Pencost,Male,21,China,Gillespie Field,US,United States,NAM,North America,2/25/2022,SEE,Bay Pencost,False,False,True


El one-hot encoding consiste en crear una variable binaria (0 o 1) por cada categoría posible de la variable original. Por ejemplo, si Flight Status tiene tres categorías: "On Time", "Delayed" y "Cancelled", se crearían tres variables: Flight Status_On Time, Flight Status_Delayed y Flight Status_Cancelled. Cada una de estas variables tomaría el valor 1 si la observación pertenece a esa categoría, y 0 si no. Así se obtienen variables numéricas que se pueden usar como predictores en el modelo de regresión lineal.

El label encoding consiste en asignar un número entero a cada categoría posible de la variable original. Por ejemplo, si Flight Status tiene tres categorías: "On Time", "Delayed" y "Cancelled", se podrían asignar los números 0, 1 y 2 respectivamente. Así se obtiene una variable numérica que se puede usar como predictor en el modelo de regresión lineal. Sin embargo, hay que tener en cuenta que esta técnica implica un orden implícito entre las categorías, lo cual puede no ser adecuado para algunas variables.

In [4]:
# Seleccionar las variables numéricas como características
X = df[['Age', 'Departure Date']]
y = df['Flight Status_On Time']

In [5]:
# Crear el modelo de regresión logística
log_model = LogisticRegression()

In [6]:
# Convertir la columna 'Departure Date' a objetos datetime
df['Departure Date'] = pd.to_datetime(df['Departure Date'], format='%m/%d/%Y')

# Convertir la columna 'Departure Date' a un número entero que representa el número de días desde el 1 de enero del año 1
df['Departure Date'] = df['Departure Date'].apply(lambda x: x.toordinal())

In [7]:
# Seleccionar las variables numéricas como características
X = df[['Age', 'Departure Date']]
y = df['Flight Status_On Time']

# Crear el modelo de regresión logística
log_model = LogisticRegression()

In [8]:
# Aplicar la validación cruzada con 5 folds
cv_results = cross_validate(log_model, X, y, cv=5, scoring=['accuracy', 'recall', 'f1', 'roc_auc'])


In [9]:
# Mostrar los resultados de cada fold
print(cv_results)

{'fit_time': array([0.07452941, 0.0483954 , 0.13202024, 0.0474112 , 0.05912066]), 'score_time': array([0.03152251, 0.03126216, 0.03525376, 0.0345335 , 0.02288198]), 'test_accuracy': array([0.66690327, 0.66695396, 0.66695396, 0.66695396, 0.66693708]), 'test_recall': array([0., 0., 0., 0., 0.]), 'test_f1': array([0., 0., 0., 0., 0.]), 'test_roc_auc': array([0.4971777 , 0.50108704, 0.49774594, 0.50461867, 0.50469113])}


**Evaluación de un modelo de aprendizaje automático utilizando validación cruzada**

fit_time: Tiempo necesario para ajustar el modelo en cada pliegue de validación.

score_time: Tiempo necesario para evaluar el modelo en cada pliegue de validación.

test_accuracy: Precisión del modelo en cada pliegue de validación. Es la proporción de predicciones correctas sobre el total de predicciones.

test_recall: Recall del modelo en cada pliegue de validación. Es la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos.

test_f1: Puntuación F1 del modelo en cada pliegue de validación. Es la media armónica de precisión y recall.

test_roc_auc: Área bajo la curva ROC (AUC) del modelo en cada pliegue de validación. Es una medida de la capacidad del modelo para distinguir entre clases positivas y negativas.

Dado que test_recall y test_f1 tienen valores de cero en todos los pliegues, parece indicar que el modelo no está prediciendo correctamente la clase positiva en ninguno de los casos. Por otro lado, la precisión (test_accuracy) es del 66.69% en todos los pliegues, lo que indica que el modelo puede estar prediciendo predominantemente la clase negativa.

La AUC ROC (test_roc_auc) varía entre alrededor de 0.497 y 0.505 en diferentes pliegues, lo que sugiere que el modelo tiene un rendimiento cercano al azar en términos de distinguir entre las clases positiva y negativa.

Estos resultados sugieren que el modelo puede estar teniendo dificultades para aprender patrones distintivos en los datos y puede requerir ajustes adicionales o un enfoque diferente para mejorar su rendimiento.

Dado que el modelo actual parece tener dificultades para distinguir entre las clases positiva y negativa, usaremos el **Ajuste de hiperparámetros** con GridSearchCV

**Búsqueda de hiperparámetros utilizando GridSearchCV**

In [10]:
# Definir el modelo SVM
svm_model = SVC()

In [11]:
# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'C': uniform(loc=0, scale=10),  # Parámetro de regularización
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Tipo de kernel
    'gamma': ['scale', 'auto'],  # Coeficiente del kernel (solo para 'rbf', 'poly', 'sigmoid')
    'degree': randint(2, 6),  # Grado del polinomio (solo para 'poly')
}

In [12]:
# Entrenar el modelo con la búsqueda aleatoria de hiperparámetros
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# Realizar la búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(
    svm_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42
)

In [14]:
# Realiza la búsqueda aleatoria
random_search.fit(X_train, y_train)

c:\Users\luciano.taddeo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\luciano.taddeo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\luciano.taddeo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

RandomizedSearchCV(cv=5, estimator=SVC(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001A383127BC0>,
                                        'degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001A383127350>,
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'poly', 'rbf',
                                                   'sigmoid']},
                   random_state=42, scoring='accuracy')

El mensaje indica que 10 de las 50 ejecuciones de ajuste del modelo fallaron con un error específico: "ValueError: The dual coefficients or intercepts are not finite. The input data may contain large values and need to be preprocessed." Esto sugiere que hay un problema con los datos de entrada que causó la falla en el ajuste del modelo SVM (Support Vector Machine). La advertencia también indica que algunas de las puntuaciones de prueba son no finitas (NaN), lo que sugiere que los resultados de algunas configuraciones de parámetros no se pudieron calcular correctamente.

In [15]:
# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", random_search.best_params_)

Mejores hiperparámetros: {'C': 3.745401188473625, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf'}


El resultado obtenido donde se han encontrado los mejores valores para los hiperparámetros del modelo. 

Aquí hay una interpretación de los hiperparámetros encontrados:

C: Este parámetro controla la penalización por error en el modelo SVM. Valores más altos de C conducen a una clasificación más precisa de los puntos de datos del conjunto de entrenamiento, pero pueden causar un sobreajuste. En este caso, el mejor valor encontrado para C es aproximadamente 3.745.

degree: Este parámetro es específico para el kernel polinómico y representa el grado del polinomio a utilizar. El grado del polinomio determina la flexibilidad del modelo. En este caso, el mejor valor encontrado para degree es 2.

gamma: Este parámetro controla la influencia de un solo ejemplo de entrenamiento, con valores más altos que indican una influencia más estrecha. scale significa que se utilizará 1 / (n_features * X.var()) como valor de gamma.

kernel: El kernel especifica el tipo de función de kernel a utilizar en el modelo SVM. Los tipos comunes de kernel incluyen 'linear', 'poly', 'rbf' (Radial Basis Function), y 'sigmoid'. En este caso, el mejor kernel encontrado es 'rbf'.

En resumen, los hiperparámetros encontrados sugieren que el modelo SVM con un kernel radial (RBF) y un valor de C alrededor de 3.745, un grado polinómico de 2 y un valor de gamma basado en escala, proporciona el mejor rendimiento para los datos y el problema específico. Estos hiperparámetros se pueden utilizar para ajustar y entrenar el modelo SVM antes de aplicarlo para hacer predicciones en nuevos datos.

In [16]:
# Mostrar la mejor puntuación
print("Mejor puntuación:", random_search.best_score_)

Mejor puntuación: 0.6676215222764433


"Mejor puntuación: 0.6676215222764433", indica la mejor puntuación obtenida durante la búsqueda de hiperparámetros para el modelo. En este caso, la puntuación parece ser un valor de métrica de evaluación del modelo, pero no se especifica qué métrica se está utilizando (por ejemplo, precisión, exactitud, AUC-ROC, etc.).

En resumen, este resultado indica la mejor puntuación obtenida durante la búsqueda de hiperparámetros, pero para una evaluación completa del modelo, se necesitaría conocer la métrica específica utilizada y considerar otros factores como el equilibrio entre sesgo y varianza, la interpretabilidad del modelo, entre otros.

In [17]:
# Obtener el mejor modelo
best_model = random_search.best_estimator_

In [18]:
# Evaluar el mejor modelo en el conjunto de prueba
test_accuracy = best_model.score(X_test, y_test)
print("Exactitud en el conjunto de prueba:", test_accuracy)

Exactitud en el conjunto de prueba: 0.6642161833299534


**Interpretación de cada uno:**

Mejor puntuación: 0.6676215222764433

Esta métrica podría ser la mejor puntuación de validación cruzada, como la precisión, el F1-score, el área bajo la curva ROC, etc. Se refiere al mejor puntaje obtenido durante el proceso de búsqueda de hiperparámetros o entrenamiento del modelo.
En este caso, parece ser un puntaje de validación cruzada o un puntaje de entrenamiento que indica el rendimiento promedio del modelo en múltiples divisiones de los datos.
Exactitud en el conjunto de prueba: 0.6642161833299534

Esta métrica indica la precisión del modelo en un conjunto de datos de prueba independiente que no se utilizó durante el entrenamiento ni la validación.
La exactitud es la proporción de predicciones correctas realizadas por el modelo sobre el conjunto de prueba.
En este caso, la exactitud del modelo en el conjunto de prueba es aproximadamente 0.664, lo que significa que el modelo clasificó correctamente alrededor del 66.4% de los ejemplos en el conjunto de prueba.

**Leave-One-Out Cross-Validation (LOOCV)**

En este enfoque, se utiliza un "fold" para la validación y todos los demás para el entrenamiento. Esto se repite para cada muestra en el conjunto de datos

In [22]:
# Crear el modelo de regresión logística
logistic_model = LogisticRegression()

In [23]:
# Aplicar Leave-One-Out Cross-Validation (LOOCV)
loo = LeaveOneOut()
loo_scores = cross_val_score(logistic_model, X, y, cv=loo)
print("Leave-One-Out Cross-Validation (LOOCV) Mean Accuracy:", loo_scores.mean())

Leave-One-Out Cross-Validation (LOOCV) Mean Accuracy: 0.6669404475810949


**5-Fold Cross-Validation**

Divide el conjunto de datos en 5 partes iguales. Se utiliza una parte para la validación y las 4 restantes para el entrenamiento. Esto se repite 5 veces, con cada parte utilizada como conjunto de validación exactamente una vez.

In [24]:
#Aplicar 5-Fold Cross-Validation
kf5 = KFold(n_splits=5)
kf5_scores = cross_val_score(logistic_model, X, y, cv=kf5)
print("5-Fold Cross-Validation Mean Accuracy:", kf5_scores.mean())

5-Fold Cross-Validation Mean Accuracy: 0.6669404398351769


**10-Fold Cross-Validation**

Similar a 5-fold, pero con 10 partes iguales. Se utiliza una parte para la validación y las 9 restantes para el entrenamiento. Esto se repite 10 veces

In [25]:
#Aplicar 10-Fold Cross-Validation
kf10 = KFold(n_splits=10)
kf10_scores = cross_val_score(logistic_model, X, y, cv=kf10)
print("10-Fold Cross-Validation Mean Accuracy:", kf10_scores.mean())

10-Fold Cross-Validation Mean Accuracy: 0.6669404705121325


**Stratified K-Fold Cross-Validation**

Similar a K-fold, pero con la garantía de que en cada "fold", la distribución de las clases objetivo sea aproximadamente la misma que en el conjunto de datos completo

In [26]:
#Aplicar Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5)
skf_scores = cross_val_score(svm_model, X, y, cv=skf)
print("Stratified K-Fold Cross-Validation Mean Accuracy:", skf_scores.mean())

Stratified K-Fold Cross-Validation Mean Accuracy: 0.666940447546933


De acuerdo a los resultados obtenidos, el modelo entrenado con los hiperparámetros mencionados logró un rendimiento consistente en diferentes técnicas de validación cruzada.